# *Импорт библиотек*

In [ ]:
%%capture
!pip install autokeras

In [ ]:
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-white')
sns.set_context('notebook', rc={'figure.figsize': (10, 8)}, font_scale=1.5)

import numpy as np # Импортируем библиотеку numpy
import pandas as pd

import time
import random
import math
import re
import os # Импортируем библиотеку os для раоты с фаловой системой
import shutil
import json
import cv2

from copy import deepcopy
from IPython.display import clear_output

# from tensorflow.keras.applications import MobileNetV3Small
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import one_hot
# from tensorflow.keras.preprocessing import image # Импортируем модуль image для работы с изображениями

from sklearn.model_selection import train_test_split

In [ ]:
from PIL import Image, ImageFile, ImageOps # импортируем модель Image для работы с изображениями
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # print('Время обработки: {:.2f} с'.format(time.time() - self.t))
        seconds_total = time.time() - self.t
        minutes = int(seconds_total // 60)
        seconds = (seconds_total % 60) + (seconds_total - math.floor(seconds_total))
        print('Время обработки: {} мин {:.2f} c'.format(minutes, seconds))

with timex():
    print()

In [ ]:
# Указываем путь к базе данных
directory = '/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans'
directory_new = '/content/ClearedReshapedBase'

# **Подготовка данных**

In [ ]:
# Загрузка датафрейма
df = pd.read_csv('/content/drive/MyDrive/df_info_new_ru.csv')

df = df[df['instant_rotation_cv2'] == 0]
df = df[df['instant_rotation_PIL'] == 0]
df.head()

In [ ]:
df.columns

In [ ]:
# Создаём выборки
# Добавляем выборку без ошибок
rows_with_zeros = df[(df[df.columns[20:]] == 0).all(axis=1)]
filtered_df = df.loc[rows_with_zeros.index]

filelist = set(list(filtered_df[filtered_df['deskew_angle'].apply(abs) < 1]['filename'])[:200])
f_sum = len(filelist)

# Добавляем выборки из разных категорий
filelist.update(set(df[df['deskew_angle'].apply(abs) > 3]['filename']))
print('{:70} {}'.format('Файлов с большим углом поворота:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist.update(set(df[df['не белый фон'] == 1]['filename']))
print('{:70} {}'.format('Не белый фон:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist.update(set(list(df[df['cropped'] == 1]['filename'])[:100]))
print('{:70} {}'.format('Обрезанные:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist.update(set(list(df[df['выброс'] == 1]['filename'])[:150]))
print('{:70} {}'.format('Выброс:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist.update(set(list(df[df['искажение документа'] == 1]['filename'])[:100]))
print('{:70} {}'.format('Искажение документа:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist.update(set(list(df[df['нестандартный контраст'] == 1]['filename'])[:100]))
print('{:70} {}'.format('Нестандартный контраст:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist.update(set(list(df[df['грязь'] == 1]['filename'])[:100]))
print('{:70} {}'.format('Грязь:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist.update(set(list(df[df['плохая печать'] == 1]['filename'])[:100]))
print('{:70} {}'.format('Плохая печать:', abs(f_sum - len(filelist))))
f_sum = len(filelist)

filelist = list(filelist)
print(f'Всего файлов для обучения: {len(filelist)}')

In [ ]:
files_dirs = []
directory = '/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans/'

for i in filelist:
    files_dirs.append(directory + i)

print(files_dirs[:3])
print(len(files_dirs))

In [ ]:
# Размер, к которому вы хотите привести изображения
target_size = (224, 224)

image_list = []
with timex():
    for i, filename in enumerate(files_dirs):
        img = cv2.imread(filename)                                          # Открываем
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                         # Переводим в чёрно-белый вид
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)    # Уменьшаем

        image_list.append(img)

        if i % 100 == 0:
            clear_output()
            print(i)

# Поворот изображений и добавление их в общую выборку
rotated_images = []
with timex():
    for img in image_list:
        rotated_images.append(img)  # Оригинальное изображение

        # Поворот на 90, -90 и 180 градусов
        for angle_func in [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]:
            rotated_images.append(cv2.rotate(img, angle_func))

del image_list

In [ ]:
rotated_images = [image / 255.0 for image in rotated_images]                    # нормализация (диапазон [0, 1])
rotated_images = [np.expand_dims(image, axis=2) for image in rotated_images]    # добавляем ещё одну размерность (224,224,1)
rotated_images = np.array(rotated_images)                                       # перевод в numpy массив

# Разметка изображений
labels = [0, 1, 2, 3] * (len(rotated_images) // 4)

# Перевод меток в формат One-Hot Encoding
one_hot_labels = np.array(one_hot(labels, depth=4))

# Создание обучающей и тестовой выборок
xTrain, xTest, yTrain, yTest = train_test_split(rotated_images, one_hot_labels, test_size=0.2, random_state=42)
del one_hot_labels, rotated_images

In [ ]:
import gc
gc.collect()

In [ ]:
print('Размер тестовой выборки: {}'.format(len(list(df[~df['filename'].isin(filelist)]['filename']))))

In [ ]:
# Тестовая выборка
filelist_val = list(df[~df['filename'].isin(filelist)]['filename'])[0:1000]
files_dirs_val = ['/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans/' + i for i in filelist_val]

target_size = (224, 224)

image_list_val = []
with timex():
    for i, filename in enumerate(files_dirs_val):
        try:
            img = cv2.imread(filename)                                          # Открываем
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                         # Переводим в чёрно-белый вид
            img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)    # Уменьшаем

            image_list_val.append(img)
        except:
            print(f'Ошибка с файлом {filename}')
            cont = input('Продолжить? (y/n) ')
            if cont == 'y':
                pass
            else:
                raise Exception()

        if i % 100 == 0:
            clear_output()
            print(i)

# Поворот изображений и добавление их в общую выборку
rotated_images_val = []
with timex():
    for img in image_list_val:
        rotated_images_val.append(img)  # Оригинальное изображение

        # Поворот на 90, -90 и 180 градусов
        for angle_func in [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]:
            rotated_images_val.append(cv2.rotate(img, angle_func))

del image_list_val

rotated_images_val = [image / 255.0 for image in rotated_images_val]                    # нормализация (диапазон [0, 1])
rotated_images_val = [np.expand_dims(image, axis=2) for image in rotated_images_val]    # добавляем ещё одну размерность (224,224,1)
rotated_images_val = np.array(rotated_images_val)                                       # перевод в numpy массив

# Разметка изображений
labels = [0, 1, 2, 3] * (len(rotated_images_val) // 4)

# Перевод меток в формат One-Hot Encoding
one_hot_labels_val = np.array(one_hot(labels, depth=4))

In [ ]:
del one_hot_labels_val, rotated_images_val

# **Обучение архитектуры autokeras**
> **Точность на валидационной выборке: 99.14%**


In [ ]:
import autokeras as ak

# Создаём ImageClassifier
clf = ak.ImageClassifier(max_trials=1)

In [ ]:
# Обучаем модель
clf.fit(xTrain, yTrain, epochs=5)

# Результаты модели
results = clf.evaluate(xTest, yTest)
print(results)

# Экспортируем модель
best_model = clf.export_model()
best_model.summary()

# Сохраняем модель
best_model.save('/content/drive/MyDrive/bestmodel_autokeras_06_12_2023.h5')

# **Дообучение модели**
> **Точность на валидационной выборке после обучения: 100.00%**

In [ ]:
import tensorflow as tf
from keras.optimizers import Adam
from keras import backend as K
from autokeras import CUSTOM_OBJECTS

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/bestmodel_autokeras_06_12_2023.h5', custom_objects=CUSTOM_OBJECTS)

In [ ]:
model.summary()

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', recall_m, precision_m, f1_m])

In [ ]:
def print_model_acc(model):
    # Оценка модели на тестовой выборке
    evaluation = model.evaluate(xTest, yTest)
    accuracy = evaluation[1]
    print(f"Точность модели на тестовой выборке: {accuracy * 100:.2f}%")

# Для сбора мусора
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

def plot_history(history):
    # Графики для precision, recall и f1
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['precision_m'], label='Precision', color='#3359FF', linestyle='-')
    plt.plot(history.history['recall_m'], label='Recall', color='#33ECFF', linestyle='--')
    plt.plot(history.history['f1_m'], label='F1-score', color='#33BCFF', linestyle='-.')

    plt.plot(history.history['val_precision_m'], label='Val Precision', color='#FF3333', linestyle='-')
    plt.plot(history.history['val_recall_m'], label='Val Recall', color='#FF8D33', linestyle='--')
    plt.plot(history.history['val_f1_m'], label='Val F1-score', color='#FFE433', linestyle='-.')

    plt.title('Precision, Recall, and F1-score')
    plt.xlabel('Epoch')
    plt.ylabel('Metrics')
    plt.legend()
    plt.grid(True)

    # График для loss и val_loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='red', linestyle='-')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='purple', linestyle='--')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

save_path = "/content/drive/MyDrive/best_model_auto.h5"

# Создание объекта ModelCheckpoint для сохранения весов
checkpoint = ModelCheckpoint(save_path, monitor='val_accuracy', save_best_only=True, mode='max')
garbage_collector = GarbageCollectorCallback()

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=64, callbacks=[checkpoint])

# Загрузка лучших весов после обучения
# model.load_weights(save_path)

In [ ]:
plot_history(history)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.save('/content/drive/MyDrive/bestmodel_autokeras_08_12_2023.h5')

# **Проверка на тестовой выборке после основного обучения**
> Проверка на 4942 файлах, и их повёрнутых версиях (итого 19768 изображений)

> **Точность: 99.77%**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/models/Qazaq Stroy | Распознание платёжных поручений/bestmodel_autokeras_08_12_2023_norm.h5', custom_objects=CUSTOM_OBJECTS)

In [ ]:
# Списки для ошибочно распознанных файлов
error_index_list = []
error_img_list = []

#### 0-1000

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 0-1000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

In [ ]:
# Делаем предикт модели
y_pred = model.predict(rotated_images_val)

# Переводим списки в классы
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(one_hot_labels_val, axis=1)

In [ ]:
# y_true - истинные метки, y_pred - предсказанные метки модели
conf_matrix = confusion_matrix(y_true, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

print()

class_report = classification_report(y_true, y_pred)
print("Отчёт о классификации:")
print(class_report)

In [ ]:
incorrect_indices = [i for i in range(len(y_true)) if y_true[i] != y_pred[i]]

print("Индексы неправильно классифицированных примеров:")
print(incorrect_indices)

In [ ]:
# Добавляем в списки
error_index_list += incorrect_indices
error_img_list += [img for i, img in enumerate(rotated_images_val) if i in incorrect_indices]

#### 1000-2000

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 1000-2000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

#### 2000-3000

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 2000-3000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

In [ ]:
# Делаем предикт модели
y_pred = model.predict(rotated_images_val)

# Переводим списки в классы
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(one_hot_labels_val, axis=1)

# y_true - истинные метки, y_pred - предсказанные метки модели
conf_matrix = confusion_matrix(y_true, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

print()

class_report = classification_report(y_true, y_pred)
print("Отчёт о классификации:")
print(class_report)

print()

incorrect_indices = [i for i in range(len(y_true)) if y_true[i] != y_pred[i]]

print("Индексы неправильно классифицированных примеров:")
print(incorrect_indices)

# Добавляем в списки
error_index_list += incorrect_indices
error_img_list += [img for i, img in enumerate(rotated_images_val) if i in incorrect_indices]

#### 3000-4000

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 3000-4000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

In [ ]:
# Делаем предикт модели
y_pred = model.predict(rotated_images_val)

# Переводим списки в классы
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(one_hot_labels_val, axis=1)

# y_true - истинные метки, y_pred - предсказанные метки модели
conf_matrix = confusion_matrix(y_true, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

print()

class_report = classification_report(y_true, y_pred)
print("Отчёт о классификации:")
print(class_report)

print()

incorrect_indices = [i for i in range(len(y_true)) if y_true[i] != y_pred[i]]

print("Индексы неправильно классифицированных примеров:")
print(incorrect_indices)

# Добавляем в списки
error_index_list += incorrect_indices
error_img_list += [img for i, img in enumerate(rotated_images_val) if i in incorrect_indices]

#### 4000-4942

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 4000-4942
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

In [ ]:
# Делаем предикт модели
y_pred = model.predict(rotated_images_val)

# Переводим списки в классы
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(one_hot_labels_val, axis=1)

# y_true - истинные метки, y_pred - предсказанные метки модели
conf_matrix = confusion_matrix(y_true, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

print()

class_report = classification_report(y_true, y_pred)
print("Отчёт о классификации:")
print(class_report)

print()

incorrect_indices = [i for i in range(len(y_true)) if y_true[i] != y_pred[i]]

print("Индексы неправильно классифицированных примеров:")
print(incorrect_indices)

# Добавляем в списки
error_index_list += incorrect_indices
error_img_list += [img for i, img in enumerate(rotated_images_val) if i in incorrect_indices]

#### Точность модели по всем тестовым выборкам

In [ ]:
# Итоговая точность
final_acc = ((0.9990 * 1000) + (1.0000 * 1000) + (0.9992 * 1000) + (0.9965 * 1000) + (0.9934 * 942)) / 4942
print(f"Точность модели на тестовой выборке: {final_acc:.2%}")

# **Дообучение на неправильно распознанных изображениях**

In [ ]:
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Составление выборок
xTrain = np.array(error_img_list)
yTrain = np.array(one_hot([i % 4 for i in error_index_list], depth=4))

In [ ]:
# Для графика точности
acc_l = []
loss_l = []

##### **Обучение**

In [ ]:
# Обучение модели
history = model.fit(xTrain, yTrain, epochs=5, batch_size=64)
acc_l += history.history['accuracy']
loss_l += history.history['loss']

In [ ]:
history = model.fit(xTrain, yTrain, epochs=5, batch_size=64)
acc_l += history.history['accuracy']
loss_l += history.history['loss']

In [ ]:
history = model.fit(xTrain, yTrain, epochs=5, batch_size=64)
acc_l += history.history['accuracy']
loss_l += history.history['loss']

In [ ]:
history = model.fit(xTrain, yTrain, epochs=5, batch_size=64)
acc_l += history.history['accuracy']
loss_l += history.history['loss']

In [ ]:
history = model.fit(xTrain, yTrain, epochs=5, batch_size=64)
acc_l += history.history['accuracy']
loss_l += history.history['loss']

##### **График обучения**

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(acc_l, label='Precision', color='#3359FF')

plt.title('Model Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)

# График для loss
plt.subplot(1, 2, 2)
plt.plot(loss_l, label='Loss', color='red')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
model.save('/content/drive/MyDrive/best_model_11_12_23.h5')

# ***Проверка на тестовой выборке дообучения***
> Проверка на 4942 файлах, и их повёрнутых версиях (итого 19768 изображений)

> **Финальная точность: 99.98%**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Списки для ошибочно распознанных файлов
error_index_list = []
error_img_list = []

#### 0-1000

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 0-1000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

#### *1000-2000*

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 1000-2000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

#### 2000-3000

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 2000-3000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

#### *3000-4000*

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 3000-4000
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

In [ ]:
# Делаем предикт модели
y_pred = model.predict(rotated_images_val)

# Переводим списки в классы
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(one_hot_labels_val, axis=1)

# y_true - истинные метки, y_pred - предсказанные метки модели
conf_matrix = confusion_matrix(y_true, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

print()

class_report = classification_report(y_true, y_pred)
print("Отчёт о классификации:")
print(class_report)

print()

incorrect_indices = [i for i in range(len(y_true)) if y_true[i] != y_pred[i]]

print("Индексы неправильно классифицированных примеров:")
print(incorrect_indices)

# Добавляем в списки
error_index_list += incorrect_indices
error_img_list += [img for i, img in enumerate(rotated_images_val) if i in incorrect_indices]

#### *4000-4942*

In [ ]:
# Оценка модели на валидационной выборке (на которой не обучали) 4000-4942
evaluation = model.evaluate(rotated_images_val, one_hot_labels_val)

accuracy = evaluation[1]
print(f"Точность модели на валидационной выборке: {accuracy * 100:.2f}%")

In [ ]:
# Делаем предикт модели
y_pred = model.predict(rotated_images_val)

# Переводим списки в классы
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(one_hot_labels_val, axis=1)

# y_true - истинные метки, y_pred - предсказанные метки модели
conf_matrix = confusion_matrix(y_true, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

print()

class_report = classification_report(y_true, y_pred)
print("Отчёт о классификации:")
print(class_report)

print()

incorrect_indices = [i for i in range(len(y_true)) if y_true[i] != y_pred[i]]

print("Индексы неправильно классифицированных примеров:")
print(incorrect_indices)

# Добавляем в списки
error_index_list += incorrect_indices
error_img_list += [img for i, img in enumerate(rotated_images_val) if i in incorrect_indices]

#### Точность модели по всем тестовым выборкам

In [ ]:
# Итоговая точность
final_acc = ((1.0000 * 1000) + (1.0000 * 1000) + (1.0000 * 1000) + (0.9995 * 1000) + (0.9995 * 942)) / 4942
print(f"Точность модели на тестовой выборке: {final_acc:.4%}")